# future_series.ipynb: 
Use the [barchart market data API](https://www.barchart.com/ondemand/free-market-data-api) to retrieve csv history files for bar sizes ranging from 1 minute to monthly.

### Use:
1. Enter values for all of the variables in section **1.1** below;
2. Run all cells below that;
3. All retrieved data will be written as csv files to the folder that you specify in the variable ```temp_folder``` in section **1.1** .

___
### Section 1.0 - Import packages and append various folder paths to sys.path so that the module futures_series.py works properly

In [ ]:
import os
import sys
import argparse as ap
import numpy as np
import pandas as pd
import datetime

WORKSPACE_DIR = '../..' #!echo $(cd ../../;pwd)
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' #!echo $(cd ../;pwd)
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' #!echo $(cd ./;pwd)
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
print(sys.path)
import futures_series as futs


In [ ]:
from importlib import reload
reload(futs)

___
### Section 1.1 - Change the contract information below to suit your needs
* Set the variable ```temp_folder``` to the folder that will hold various temporary data, and will also hold 2 text files:
 * free_api_key.txt
 * paid_api_key.txt
 * Both free and paid api_key.txt files should hold a 32 digit API key (like 0123456789abcdefghij0123456789ab) that you obtain from barchart
* Set the variable ```commodity``` to determine the commodity futures contracts that you want to obtain (e.g, CL, ES, NG: see the [barchart commodity codes](https://www.barchart.com/futures/most-active/all?viewName=main) )
* Set the variable ```start_year``` to determine the first year from which you want to retrieve data
* Set the variable ```end_year``` to determine the last year from which you want to retrieve data
 * _For each year, you will retrieve history data for all of the contract months that traded in that year_
* Set the variable ```days``` to determine the number of days per contract in all the contract months that you will retrieve 
* Set the variable ```bar_type``` to determine the time scale of the csv data that you want (minutes, daily, monthly)
* Set the variable ```interval``` to determine the number of minutes that you want for each "minutes" bar.  
 * _Set this to 1 for daily and monthly bar_type's_

In [8]:
# api_key_type_prefix = 'paid'
api_key_type_prefix = 'free'
temp_folder = './temp_folder'
api_key = open(f'{temp_folder}/{api_key_type_prefix}_api_key.txt','r').read()
commodity = 'CL'
start_year = 2008
end_year = 2018
days = 40
# bar_type = 'minutes'
bar_type = 'daily'
interval = 1

#### (Keep these the same)

In [9]:
args = ap.Namespace(
    output_folder = temp_folder,
    endpoint_type = f'{api_key_type_prefix}_url'
)
quarter_commodities = ['ES','GE','GC']

___
### Section 2.0

The code below iterates through each year, calling 
```
    FuturesSeries.get_contracts(trading_days_to_get=days,month_code_list=month_codes)
    FuturesSeries.write_csv(output_folder)
```
in order to retrieve and save all of the futures contracts for each year.

In [12]:
# bar_type = args.bar_type
# interval = args.interval
output_folder = temp_folder
logger = futs.init_root_logger('logger.log')
endpoint_type = args.endpoint_type

logger.info(f'start')
# start_year = 2008
# end_year = 2018
years = np.linspace(start_year,end_year,end_year-start_year+1,dtype=int)
# days = 40
month_codes = None
if commodity in quarter_commodities:
    month_codes = ['H','M','U','Z']
    days = 120
logger.info(f'days,month_codes {(days,month_codes)}')
for y in years:
    beg_yyyymm = int(f'{y}01')
    end_yyyymm = int(f'{y}12')
    logger.info(f'begin:{beg_yyyymm}, end:{end_yyyymm}')
    fc = futs.FuturesSeries(api_key, commodity, beg_yyyymm, end_yyyymm, bar_type, 
                            interval, endpoint_type,logger)
    logger.info(f'BEGIN Fetching Data for year: {y}')
    try:
        fc.get_contracts(trading_days_to_get=days,month_code_list=month_codes)
        logger.info(f'BEGIN Writing Data for year: {y}')
        fc.write_csv(output_folder)
        logger.info(f'END Writing Data for year: {y}')
    except Exception as e:
        logger.info(f' error getting contract {commodity} for begin:{str(beg_yyyymm)} and end:{str(end_yyyymm)} {str(e)}')
logger.info(f'end')



2019-01-08 11:03:18,649 - root - INFO - start
2019-01-08 11:03:18,651 - root - INFO - days,month_codes (40, None)
2019-01-08 11:03:18,653 - root - INFO - begin:200801, end:200812
2019-01-08 11:03:18,654 - root - INFO - BEGIN Fetching Data for year: 2008
  0%|          | 0/12 [00:00<?, ?it/s]2019-01-08 11:03:18,662 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): marketdata.websol.barchart.com


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


2019-01-08 11:03:20,263 - requests.packages.urllib3.connectionpool - DEBUG - https://marketdata.websol.barchart.com:443 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20071201&symbol=CLF08&type=daily&apikey=99125d367af268ca234144d64583a5e7 HTTP/1.1" 200 85

2019-01-08 11:03:20,271 - root - INFO -  error getting contract CL for begin:200801 and end:200812 single positional indexer is out-of-bounds
2019-01-08 11:03:20,272 - root - INFO - begin:200901, end:200912
2019-01-08 11:03:20,273 - root - INFO - BEGIN Fetching Data for year: 2009
  0%|          | 0/12 [00:00<?, ?it/s]2019-01-08 11:03:20,281 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): marketdata.websol.barchart.com


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


2019-01-08 11:03:22,404 - requests.packages.urllib3.connectionpool - DEBUG - https://marketdata.websol.barchart.com:443 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20081201&symbol=CLF09&type=daily&apikey=99125d367af268ca234144d64583a5e7 HTTP/1.1" 200 85

2019-01-08 11:03:22,409 - root - INFO -  error getting contract CL for begin:200901 and end:200912 single positional indexer is out-of-bounds
2019-01-08 11:03:22,411 - root - INFO - begin:201001, end:201012
2019-01-08 11:03:22,412 - root - INFO - BEGIN Fetching Data for year: 2010
  0%|          | 0/12 [00:00<?, ?it/s]2019-01-08 11:03:22,421 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): marketdata.websol.barchart.com


...

2019-01-08 11:03:36,816 - requests.packages.urllib3.connectionpool - DEBUG - https://marketdata.websol.barchart.com:443 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20151201&symbol=CLF16&type=daily&apikey=99125d367af268ca234144d64583a5e7 HTTP/1.1" 200 85

2019-01-08 11:03:36,826 - root - INFO -  error getting contract CL for begin:201601 and end:201612 single positional indexer is out-of-bounds
2019-01-08 11:03:36,827 - root - INFO - begin:201701, end:201712
2019-01-08 11:03:36,828 - root - INFO - BEGIN Fetching Data for year: 2017
  0%|          | 0/12 [00:00<?, ?it/s]2019-01-08 11:03:36,837 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): marketdata.websol.barchart.com


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


2019-01-08 11:03:38,697 - requests.packages.urllib3.connectionpool - DEBUG - https://marketdata.websol.barchart.com:443 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20161201&symbol=CLF17&type=daily&apikey=99125d367af268ca234144d64583a5e7 HTTP/1.1" 200 85

2019-01-08 11:03:38,703 - root - INFO -  error getting contract CL for begin:201701 and end:201712 single positional indexer is out-of-bounds
2019-01-08 11:03:38,705 - root - INFO - begin:201801, end:201812
2019-01-08 11:03:38,707 - root - INFO - BEGIN Fetching Data for year: 2018
  0%|          | 0/12 [00:00<?, ?it/s]2019-01-08 11:03:38,718 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): marketdata.websol.barchart.com


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


2019-01-08 11:03:41,156 - requests.packages.urllib3.connectionpool - DEBUG - https://marketdata.websol.barchart.com:443 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20171201&symbol=CLF18&type=daily&apikey=99125d367af268ca234144d64583a5e7 HTTP/1.1" 200 85

2019-01-08 11:03:41,162 - root - INFO -  error getting contract CL for begin:201801 and end:201812 single positional indexer is out-of-bounds
2019-01-08 11:03:41,164 - root - INFO - end


## End